In [1]:
from JQLBuilder import JQLBuilder
from Enums import Enums
from JiraFacade import JiraFacade

In [59]:
jql = JQLBuilder().set_project(Enums.project_shopping).set_sprints_from_squads(Enums.squad_name_discovery, '03', '3').get()

In [60]:
df = JiraFacade().get_df_from_jql(jql, 100)

# out of plan report

In [ ]:
out_of_plan_report = df.groupby(['team_name', 'is_out_of_plan']).agg({'estimate': ['sum', 'count']}).reset_index()

out_of_plan_report

# Status Report

In [64]:
delivered_statuses = [
    'Ready To Test',
    Enums.status_testing,
    'Test Passed',
    'Ready To Deploy',
    Enums.status_deployed,
    Enums.status_done,
    Enums.status_closed
]

remaining_statuses = [
    'In Progress',
    'Sprint Backlog',
    'Code Review',
]

revision_or_blocked_statuses = [
    Enums.status_blocked,
    Enums.status_revision,
]

In [ ]:
def get_new_status(status: str):
    if status in delivered_statuses:
        return 'delivered'
    if status in remaining_statuses:
        return 'remained'
    if status in revision_or_blocked_statuses:
        return 'revision_or_blocked'
    return 'unknown'


status_report = df
status_report['new_status'] = df.apply(lambda x: get_new_status(x['status']), axis=1)
status_report = status_report.groupby(['team_name', 'new_status']).agg({'estimate': ['count', 'sum']}).reset_index()
status_report.pivot(index='new_status', columns='team_name')

In [ ]:
detailed_status_report = df.groupby(['team_name', 'status']).agg({'key': 'count'}).reset_index()
detailed_status_report = detailed_status_report.pivot(index='status', columns='team_name', values='key')
detailed_status_report